In [1]:
from clock_project.genome_analysis.bootstrapping.clock_violation_bootstrapping import clock_bootstrapper
from clock_project.genome_analysis.bootstrapping.non_stationarity_bootstrapping import bootstrapper

from cogent3 import get_app, open_data_store
from cogent3.app import io as io_app

def get_id(result):
    return result.source.unique_id

loader = get_app("load_aligned", moltype="dna")
loader_db = io_app.load_db()

In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import uniform

def qq_plot_uniform(data, a=0, b=1):
    """
    Creates a QQ plot of the provided data against a uniform distribution using Plotly.

    Args:
    data (array-like): The dataset to plot.
    a (float): The lower bound of the uniform distribution (default 0).
    b (float): The upper bound of the uniform distribution (default 1).
    """
    # Scale data for the specified uniform range
    data = np.array(data)
    data.sort()  # Sort the data for plotting
    scaled_data = (data - a) / (b - a)

    # Calculate quantiles
    n = len(data)
    theoretical_quantiles = uniform.ppf(np.arange(1, n + 1) / (n + 1))

    # Create a QQ plot
    fig = go.Figure()

    # Adding scatter plot for QQ plot
    fig.add_trace(go.Scatter(x=theoretical_quantiles, y=scaled_data, mode='markers',
                             name=None,
                             marker=dict(color='#67a8cd', size = 5)))

    # Add line of perfect fit
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                             name=None,
                             line=dict(color='red', dash='dash')))
    
    
    fig.update_layout(title=None, 
                  xaxis_title='Uniform Quantiles', 
                  yaxis_title=r'$\hat{p}-\text{value}$',
                    template='plotly_white',
                    width=500, height=600,
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 25},  
                    xaxis_title_font={'size': 25},
                    showlegend=True,
                        xaxis=dict(
            title_font=dict(size=25, family='Arial', color='black'),
            tickfont=dict(size=14),
            gridcolor='lightgrey'
        ),
                    yaxis=dict(
            title_font=dict(size=25, family='Arial', color='black'),
            tickfont=dict(size=14),
            gridcolor='lightgrey')
    )

    return fig


def qq_plot_null_observed(data, data2, a=0, b=1):
    """
    Creates a QQ plot of the provided data against a uniform distribution using Plotly.

    Args:
    data (array-like): The dataset to plot.
    a (float): The lower bound of the uniform distribution (default 0).
    b (float): The upper bound of the uniform distribution (default 1).
    """
    # Scale data for the specified uniform range
    data = np.array(data)
    data.sort()  # Sort the data for plotting
    scaled_data = (data - a) / (b - a)

    # Calculate quantiles
    n = len(data)
    theoretical_quantiles = uniform.ppf(np.arange(1, n + 1) / (n + 1))

    # Scale data for the specified uniform range
    data2 = np.array(data2)
    data2.sort()  # Sort the data for plotting
    scaled_data2 = (data2 - a) / (b - a)

    # Calculate quantiles
    n2 = len(data2)
    theoretical_quantiles2 = uniform.ppf(np.arange(1, n2 + 1) / (n2 + 1))

    # Create a QQ plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=theoretical_quantiles, y=scaled_data, mode='markers',
                                name='<b>-ve</b>',
                                marker=dict(color='#67a8cd', size = 4)))

    # Adding scatter plot for QQ plot
    fig.add_trace(go.Scatter(x=theoretical_quantiles2, y=scaled_data2, mode='markers',
                                name='<b>Observed</b>',
                                marker=dict(color='#6fba4f', size = 4)))

    # # Add line of perfect fit
    # fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
    #                             name=None,
    #                             line=dict(color='red', dash='dash')))

    fig.update_layout(title=None, xaxis_title='Uniform Quantiles', yaxis_title=r'$\hat{p}-\text{value}$',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 25, 'family': 'Arial', 'color': 'black'},  
                    xaxis_title_font={'size': 25}, 
                    width=600, height=600,
                    showlegend=True,
                    legend=dict(font = dict(size=16)),
            xaxis=dict(
            title_font=dict(size=25, family='Arial', color='black'),
            tickfont=dict(size=14),
            gridcolor='lightgrey'
        ),
                    yaxis=dict(
            title_font=dict(size=25, family='Arial', color='black'),
            tickfont=dict(size=14),
            gridcolor='lightgrey'))
    
    
    return fig

In [2]:
triples_subset_path = '/Users/gulugulu/clock/mammal_orthologs_hsap_1/triples_representative_subset.sqlitedb'


path_to_dir = '/Users/gulugulu/clock/mammal_orthologs_hsap_1/test_bootstrap_clock'
out_dstore = open_data_store(path_to_dir, mode="w", suffix="json")
write_json_app = get_app("write_json", data_store=out_dstore, id_from_source = get_id)

input_data_store = open_data_store(triples_subset_path)



In [3]:
clock_bootstrap_process = loader_db + clock_bootstrapper + write_json_app


In [11]:
clock_bootstrap_result_dir = open_data_store('/Users/gulugulu/clock/mammal_orthologs_hsap_1/test_bootstrap_clock', suffix='json')


In [ ]:
clock_bootstrap_process.apply_to(
        input_data_store,
        parallel=False,
    )
print("Finish app call")


In [30]:
clock_bootstrap_process.disconnect()

In [ ]:
load_json_app = get_app("load_json")
clock_boostrap_result_example = load_json_app('/Users/gulugulu/clock/mammal_orthologs_hsap_1/test_bootstrap_clock/ENSG00000137962_257.json')
clock_boostrap_result_example.observed

Statistics
======================
    LR    df    pvalue
----------------------
0.0012     1    0.9722
----------------------
=================================================================
hypothesis    key                  lnL    nfp    DLC     unique_Q
-----------------------------------------------------------------
null          'clock'       -3443.9654     16    True    True    
alt           'no-clock'    -3443.9648     17    True    True    
-----------------------------------------------------------------

In [31]:
non_bootstrap_process = loader_db + bootstrapper + write_json_app


ValueError: test_hypothesis_model_bootstrapper already part of composed function, use disconnect() to free them up